In [ ]:
import sys;
sys.path.insert(0, '..')

In [ ]:
from math import log2
import numpy as np

from sim_circuit import QuantumRegister, QuantumCircuit


def phase_estimation_unitary(n, U, swap=True):
    assert(U.shape[0] == U.shape[1])
    m = int(log2(U.shape[0]))

    eigvals, eigvecs = np.linalg.eig(U)

    q = QuantumRegister(n)
    a = QuantumRegister(m)
    qc = QuantumCircuit(q, a) # ancilla is last

    qc.append_u(eigvecs, a)
    qc.report('eigenstate')

    for i in range(n):
        qc.h(q[i])

    for i in range(n):
        if swap:
            for _ in range(2**i):
                qc.c_append_u(U, q[i], a)
        else:
            # qubit reversal
            for _ in range(2**i):
                qc.c_append_u(U, q[n-1-i], a)
            # decreasing powers of 2
            # for _ in range(2**(n-1-i)):
            #     qc.c_append_u(U, q[i], a)

    qc.report('geometric_sequence_superposition')

    qc.append_u(np.conj(eigvecs.transpose()), a)

    qc.report('geometric_sequence')

    qc.iqft(q if swap else q[::-1], swap)
    qc.report('estimate')

    return qc

In [ ]:
import scipy.stats
    
n = 3
m = 2
    
U = scipy.stats.unitary_group.rvs(2**m)
qc = phase_estimation_unitary(n, U, swap=True)

In [ ]:
from math import pi

eigvals, _ = np.linalg.eig(U)
theta = np.angle(eigvals[0])
if theta < 0:
    theta += 2*pi

v = theta/pi*2**(n-1)
print('\nfrequency from eigenvalue', v)

In [ ]:
from util import print_state_table

state = qc.run()
print_state_table(qc.run())

In [ ]:
result = qc.measure(shots = 1000)

sorted_counts = sorted(result['counts'].items(), key = lambda item: item[1], reverse=True)
sorted_counts

In [ ]:
from math import sqrt 

top_two = sorted(sorted_counts[:2])

p_below, p_above = top_two[0][1], top_two[1][1]

decimal_estimate = sqrt(p_above)/(sqrt(p_below)+ sqrt(p_above))

estimate = top_two[0][0] + decimal_estimate
print('\nfrequency from eigenvalue', v, '\nfrequency from measurement', estimate, '\nerror', abs(v - estimate))

In [ ]:
from util import all_close, cis, prod
from math import cos

def complex_sincd(n, v):
    N = 2 ** n
    return [prod(
        cos((v - k) * pi / 2 ** (j + 1)) * cis((v - k) * pi / 2 ** (j + 1))
        for j in range(n)) for k in range(2 ** n)]


assert all_close(state, complex_sincd(n, v))

In [ ]:
def test_unitary_inverse():
    n = 3
    m = 2
    
    U = scipy.stats.unitary_group.rvs(2**m)
    qc = phase_estimation_unitary(n, U, swap=True)
    qci = qc.inverse()

    qc.append(qci, QuantumRegister(m+n))
    state = qc.run()
    tabulate_state(state)

    assert all_close(state, init_state(m+n))